Using a LSTM single model to text various cleaning steps and impact on score.

Controls:
- CNN single model
- maxlen: 65
- min occurance vocab: 5
- glove.6B.100D
- epochs: 2
- cv: 3
- max features 20000

In [1]:
import os
import logging

In [2]:
model_name = 'grid_benchmark'
logging.basicConfig(filename=model_name+'.log',level=logging.DEBUG)

In [3]:
dir_path = os.path.realpath('..')

In [4]:
# Import custom transformers

path = 'src/features'
full_path = os.path.join(dir_path, path)
import sys
sys.path.append(full_path)
from transformers import TextCleaner, KerasProcesser

Using TensorFlow backend.


## Import data

In [5]:
import numpy as np
import pandas as pd

In [6]:
path = 'data/raw/train.csv'

full_path = os.path.join(dir_path, path)
df_train = pd.read_csv(full_path, header=0, index_col=0)
print("Dataset has {} rows, {} columns.".format(*df_train.shape))

Dataset has 95851 rows, 7 columns.


In [7]:
# fill NaN with string "unknown"
df_train.fillna('unknown',inplace=True)

## Pre-processing

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
seed = 42
np.random.seed(seed)
test_size = 0.2
target = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
corpus = 'comment_text'

X = df_train[corpus]
y = df_train[target]


Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=test_size, random_state=seed)

In [10]:
max_features=20000
max_length=65

## Model fit

In [11]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.pipeline import Pipeline
from keras.callbacks import CSVLogger

In [12]:
from keras.models import Sequential
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation

# Function to create model, required for KerasClassifier
def create_model(optimizer='adam', max_features=max_features, max_length=max_length):
    model = Sequential()
    model.add(Embedding(max_features, 100, input_length=max_length))
    model.add(Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1)))
    model.add(GlobalMaxPool1D())
    model.add(Dense(50, activation="relu"))
    model.add(Dropout(0.1))
    model.add(Dense(6, activation='sigmoid'))  #multi-label (k-hot encoding)
    # compile network
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [13]:
def save_model(model, model_path):
    # serialize model to JSON
    model_json = model.to_json()
    with open(model_path + ".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(model_path + ".h5")
    print("Saved model to disk")

In [18]:
csv_logger = CSVLogger(model_name+'.csv', append=True, separator=';')
model = KerasClassifier(build_fn=create_model, epochs=3, verbose=2)

In [23]:
p = Pipeline([
    ('cleaner', TextCleaner()),
    ('keraser', KerasProcesser(num_words=max_features, maxlen=max_length)),
    ('clf', model)
])

param_grid = {"cleaner__regex": ['\S+'],
              "cleaner__remove_digits": [True, False],
              "cleaner__english_only": [True, False],
              "cleaner__stop_words": [None],
              "cleaner__filters": [r'[!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n]'],
              "cleaner__lower": [True, False],
              "keraser__num_words": [max_features],
              "keraser__maxlen": [max_length],
              "clf__callbacks": [[csv_logger]]
             }

In [25]:
%%time
grid = GridSearchCV(p, param_grid=param_grid, cv=3)
grid_result = grid.fit(Xtrain, ytrain)

Epoch 1/3
 - 2s - loss: 0.6942 - acc: 0.4744
Epoch 2/3
 - 0s - loss: 0.6864 - acc: 0.7692
Epoch 3/3
 - 0s - loss: 0.6792 - acc: 0.8141
Epoch 1/3
 - 3s - loss: 0.6956 - acc: 0.4753
Epoch 2/3
 - 0s - loss: 0.6872 - acc: 0.7469
Epoch 3/3
 - 0s - loss: 0.6790 - acc: 0.8889
Epoch 1/3
 - 3s - loss: 0.6993 - acc: 0.3086
Epoch 2/3
 - 0s - loss: 0.6898 - acc: 0.6420
Epoch 3/3
 - 0s - loss: 0.6813 - acc: 0.9630
Epoch 1/3
 - 3s - loss: 0.6943 - acc: 0.4750
Epoch 2/3
 - 0s - loss: 0.6852 - acc: 0.8792
Epoch 3/3
 - 0s - loss: 0.6764 - acc: 0.8833
CPU times: user 34.7 s, sys: 3.21 s, total: 37.9 s
Wall time: 31.4 s


In [26]:
trained_model = grid_result.best_estimator_.named_steps['clf'].model

In [27]:
# save the model
model_path = os.path.join(dir_path, 'models', model_name)
save_model(trained_model, model_path)

Saved model to disk


## Evaluation

In [28]:
# summarize results
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
times = grid_result.cv_results_['mean_fit_time']
for mean, stdev, param, time in zip(means, stds, params, times):
    print("%f (%f) with: %r in %f seconds" % (mean, stdev, param, time))
    logging.info("%f (%f) with: %r in %f seconds" % (mean, stdev, param, time))
    
print("Best score {} with params {}".format(grid_result.best_score_, grid_result.best_params_))
logging.info("Best score {} with params {}".format(grid_result.best_score_, grid_result.best_params_))

0.929167 (0.070512) with: {'keraser__num_words': 20000, 'cleaner__remove_digits': False, 'cleaner__regex': '\\S+', 'cleaner__filters': '[!"#$%&()*+,-./:;<=>?@[\\\\]^_`{|}~\\t\\n]', 'cleaner__english_only': False, 'keraser__maxlen': 65, 'clf__callbacks': [<keras.callbacks.CSVLogger object at 0x1a1fb5b0f0>], 'cleaner__lower': True, 'cleaner__stop_words': None} in 6.943988 seconds
Best score 0.9291666522622108 with params {'keraser__num_words': 20000, 'cleaner__remove_digits': False, 'cleaner__regex': '\\S+', 'cleaner__filters': '[!"#$%&()*+,-./:;<=>?@[\\\\]^_`{|}~\\t\\n]', 'cleaner__english_only': False, 'keraser__maxlen': 65, 'clf__callbacks': [<keras.callbacks.CSVLogger object at 0x1a1fb5b0f0>], 'cleaner__lower': True, 'cleaner__stop_words': None}
